# Домашнее задание №9 к лекции «Улучшение качества модели»

Взять boston house-prices datase (sklearn.datasets.load_boston). Возмите 7 любых регрессоров (попробовать разные алгоритмы, поподбирать параметры, вывести итоговое качество).

In [39]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
data = pd.read_csv(load_boston()['filename'], skiprows=1)

In [2]:
data

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48,22.0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    int64  
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    int64  
 9   TAX      506 non-null    int64  
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(11), int64(3)
memory usage: 55.5 KB


Нет пропущенных значений

Используем следующие регрессоры:
- линейная регрессия
- регрессия дерева решений
- регрессия LASSO
- Гребневая регрессия (ридж-регрессия)
- Регрессия ElasticNet
- Регрессия кернел-риджа
- Bayesian Ridge Regression

In [70]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import ElasticNetCV
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import BayesianRidge

In [36]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

In [5]:
X = data[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT']]
y = data[['MEDV']]

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42 )  

In [71]:
models=[
      {'name':'Lr',"model": LinearRegression()  , 'params':{'fit_intercept':['False', 'True'], 'normalize':['False', 'True']}},
      {'name':'DTR',"model":DecisionTreeRegressor(), 'params':{'criterion':['mse', 'friedman_mse', 'mae', 'poisson']}},  
      {'name':'LassoCV',"model": LassoCV(), 'params':{'normalize': ['False', 'True'], 'n_alphas':[100, 200, 500]}},
      {'name':'RidgeCV',"model": RidgeCV(), 'params':{'fit_intercept':['False', 'True'], 'gcv_mode':['auto', 'svd', 'eigen']}},
      {'name':'ElasticNetCV',"model": ElasticNetCV(), 'params':{'l1_ratio':[0.1, 0.2, 0.3, 0.4, 0.5, 0.7, 0.9], 'selection':['cyclic', 'random']}},
      {'name':'KernelRidge',"model": KernelRidge(), 'params':{'kernel':['linear', 'rbf'], 'alpha':[1e-3, 5e-4, 1, 0.3], 'gamma':[0.001, 0.01, 0.005, 0.1, 0.5]}},
      {'name':'BayesianRidge',"model": BayesianRidge(), 'params':{'n_iter':[100, 200, 300, 400], 'tol':[1e-3, 5e-4, 0.02, 0.3, 0.5, 1], 'compute_score':['False', 'True']}}, 
]

res=[]
for v in  models:
    res.append((v['name'], GridSearchCV(v['model'], v['params'], cv=10).fit(X_train, np.ravel(y_train))))
    
#     'scoring':['string', 'callable']

In [72]:
for r in res:
    print(r[0], r[1].best_score_, r[1].best_params_)

Lr 0.6875346951141132 {'fit_intercept': 'False', 'normalize': 'False'}
DTR 0.6862453934304573 {'criterion': 'friedman_mse'}
LassoCV 0.68320894276593 {'n_alphas': 500, 'normalize': 'False'}
RidgeCV 0.6874165472581465 {'fit_intercept': 'False', 'gcv_mode': 'auto'}
ElasticNetCV 0.666825125329554 {'l1_ratio': 0.9, 'selection': 'cyclic'}
KernelRidge 0.6698711296169739 {'alpha': 1, 'gamma': 0.001, 'kernel': 'linear'}
BayesianRidge 0.6783295832316307 {'compute_score': 'False', 'n_iter': 100, 'tol': 0.0005}
